# Symmetric Encryption: AES Algorithm

## Contents
1. AES Algorithm Overview
2. Encryption & Decryption Implementation
3. How AES Works
4. Attack Scenarios
5. Symmetric vs Asymmetric Comparison

## Prerequisites

```bash
pip install cryptography pycryptodome
```

In [58]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import rsa, padding as asym_padding
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from Crypto.Util.Padding import pad, unpad
import os
import time
import hashlib

## 1. AES Algorithm Overview

**AES (Advanced Encryption Standard)** = Most widely used symmetric encryption algorithm

### Key Features:
- **Adopted:** 2001 by NIST (replaced DES)
- **Type:** Block cipher (encrypts fixed-size blocks)
- **Block size:** 128 bits (16 bytes)
- **Key sizes:** 128, 192, or 256 bits
- **Rounds:** 10 (AES-128), 12 (AES-192), 14 (AES-256)

### Common Modes:
- **ECB** (Electronic Codebook) - Insecure, not recommended
- **CBC** (Cipher Block Chaining) - Requires IV
- **CTR** (Counter) - Parallelizable
- **GCM** (Galois/Counter Mode) - Authenticated encryption (recommended)

### Use Cases:
- File/disk encryption
- VPN tunnels
- Wireless security (WPA2/3)
- Database encryption
- TLS/SSL (after key exchange)

## 2. Encryption & Decryption Implementation

### AES-256 with CBC Mode

In [59]:
def aes_encrypt(plaintext, key):
    """
    Encrypt data using AES-256-CBC
    """
    cipher = AES.new(key, AES.MODE_CBC)
    ciphertext = cipher.encrypt(pad(plaintext.encode('utf-8'), AES.block_size))
    return cipher.iv + ciphertext  # Prepend initial vector to ciphertext

def aes_decrypt(ciphertext, key):
    """
    Decrypt data using AES-256-CBC
    """
    iv = ciphertext[:16]  # Extract initial vector
    ct = ciphertext[16:]
    cipher = AES.new(key, AES.MODE_CBC, iv)
    plaintext = unpad(cipher.decrypt(ct), AES.block_size)
    return plaintext.decode('utf-8')

# Generate 256-bit key
key = get_random_bytes(32)  # 32 bytes = 256 bits

print("AES-256 ENCRYPTION EXAMPLE")
print("="*70)
print(f"Key (hex): {key.hex()}")
print(f"Key size: {len(key) * 8} bits\n")

# Original message
message = "This is a secret message encrypted with AES-256!"
print(f"Original message: {message}")
print(f"Length: {len(message)} characters\n")

# Encrypt
start = time.time()
encrypted = aes_encrypt(message, key)
enc_time = time.time() - start

print(f"Encrypted (hex): {encrypted.hex()}")
print(f"Length: {len(encrypted)} bytes")
print(f"Encryption time: {enc_time*1000:.4f}ms\n")

# Decrypt
start = time.time()
decrypted = aes_decrypt(encrypted, key)
dec_time = time.time() - start

print(f"Decrypted message: {decrypted}")
print(f"Decryption time: {dec_time*1000:.4f}ms")
print(f"\nMatch: {message == decrypted}")

AES-256 ENCRYPTION EXAMPLE
Key (hex): 0bd77be5977d2d48b3f4bedc58d8070333f18432138dc7ed8c65a3e78a37bff5
Key size: 256 bits

Original message: This is a secret message encrypted with AES-256!
Length: 48 characters

Encrypted (hex): 397c014436e003cfa85427dbd66c1e43a865d4223b3fd5e29f7fa83fe9c61bd5e53646596c00af6a3f9f8bb4cb0e1b638d3f3a96c7f3711534252f6e7076df13898822c17a586ec49eb4e8d3c12dc43c
Length: 80 bytes
Encryption time: 0.0000ms

Decrypted message: This is a secret message encrypted with AES-256!
Decryption time: 0.0000ms

Match: True


### AES-256 with GCM Mode (Authenticated Encryption)

In [60]:
def aes_gcm_encrypt(plaintext, key):
    """
    Encrypt with AES-256-GCM (provides authentication)
    """
    cipher = AES.new(key, AES.MODE_GCM)
    ciphertext, tag = cipher.encrypt_and_digest(plaintext.encode('utf-8'))
    return cipher.nonce + tag + ciphertext

def aes_gcm_decrypt(ciphertext, key):
    """
    Decrypt with AES-256-GCM
    """
    nonce = ciphertext[:16]
    tag = ciphertext[16:32]
    ct = ciphertext[32:]
    cipher = AES.new(key, AES.MODE_GCM, nonce=nonce)
    plaintext = cipher.decrypt_and_verify(ct, tag)
    return plaintext.decode('utf-8')

print("AES-256-GCM (AUTHENTICATED ENCRYPTION)")
print("="*70)

message = "Authenticated and encrypted message"
key_gcm = get_random_bytes(32)

print(f"Original: {message}\n")

# Encrypt
encrypted_gcm = aes_gcm_encrypt(message, key_gcm)
print(f"Encrypted (hex): {encrypted_gcm.hex()[:80]}...")
print(f"Length: {len(encrypted_gcm)} bytes\n")

# Decrypt
decrypted_gcm = aes_gcm_decrypt(encrypted_gcm, key_gcm)
print(f"Decrypted: {decrypted_gcm}")
print(f"Match: {message == decrypted_gcm}\n")

# Test tampering detection
print("Testing tampering detection...")
tampered = bytearray(encrypted_gcm)
tampered[-1] ^= 0x01  # Flip one bit

try:
    aes_gcm_decrypt(bytes(tampered), key_gcm)
    print("Tampering not detected (shouldn't happen)")
except ValueError:
    print("Tampering detected - decryption failed!")
    print("GCM provides both encryption and integrity")

AES-256-GCM (AUTHENTICATED ENCRYPTION)
Original: Authenticated and encrypted message

Encrypted (hex): 73c6bc6da9b90ea966478d885ad4f499d9124af210ea9ff21cdcf7cadb8c2d15037cb7db22b73440...
Length: 67 bytes

Decrypted: Authenticated and encrypted message
Match: True

Testing tampering detection...
Tampering detected - decryption failed!
GCM provides both encryption and integrity


## 3. How AES Works

### AES Structure (Rijndael Algorithm):

AES operates on a 4×4 matrix of bytes (128-bit block) through multiple rounds.

### Each Round Consists of:

1. **SubBytes** - Non-linear substitution using S-box
   - Each byte replaced with another (confusion)
   - Protects against linear cryptanalysis

2. **ShiftRows** - Cyclically shift rows
   - Row 0: no shift
   - Row 1: shift left by 1
   - Row 2: shift left by 2
   - Row 3: shift left by 3

3. **MixColumns** - Mix column data
   - Mathematical operation in GF(2^8)
   - Provides diffusion
   - (Skipped in final round)

4. **AddRoundKey** - XOR with round key
   - Derived from main key via key schedule
   - Different key each round

### Key Expansion:
- Main key expanded into separate round keys
- Uses Rijndael's key schedule algorithm
- Ensures each round has unique key material

## AES Encryption Process

Input: 128-bit plaintext block + 256-bit key

Initial Round:
  → AddRoundKey (XOR with key)

Rounds 1-13:
  1. SubBytes    (S-box substitution)
  2. ShiftRows   (Row permutation)
  3. MixColumns  (Column mixing)
  4. AddRoundKey (XOR with round key)

Final Round (14):
  1. SubBytes
  2. ShiftRows
  3. AddRoundKey (no MixColumns)

Output: 128-bit ciphertext block

KEY PROPERTIES:
  - Confusion: S-box makes relationship complex
  - Diffusion: Each input bit affects many output bits
  - Security: No practical attacks on full AES
  - Performance: Fast in hardware and software

## 4. Attack Scenarios

### Attack Method 1: Brute Force

In [61]:
print("ATTACK 1: BRUTE FORCE")
print("="*70)
print("""
Concept: Try all possible keys until correct one found

Key Space:
  • AES-128: 2^128 = 3.4 x 10^38 keys
  • AES-192: 2^192 = 6.3 x 10^57 keys
  • AES-256: 2^256 = 1.1 x 10^77 keys

Feasibility:
  Assume 1 billion keys/second tested:
  • AES-128: ~10^21 years to crack
  • AES-256: ~10^51 years (longer than universe age)

Status: INFEASIBLE
Mitigation: Use AES-128 minimum (AES-256 for high security)
""")

# Demonstrate small key space (for illustration only)
print("\nDemonstration with 16-bit key (weak):")
weak_key_space = 2**16
print(f"  Key space: {weak_key_space:,} keys")
print(f"  Time at 1M keys/sec: {weak_key_space/1000000:.2f} seconds")
print("\n✗ 16-bit key is INSECURE")
print("✓ AES-256: 2^256 keys = SECURE")

ATTACK 1: BRUTE FORCE

Concept: Try all possible keys until correct one found

Key Space:
  • AES-128: 2^128 = 3.4 x 10^38 keys
  • AES-192: 2^192 = 6.3 x 10^57 keys
  • AES-256: 2^256 = 1.1 x 10^77 keys

Feasibility:
  Assume 1 billion keys/second tested:
  • AES-128: ~10^21 years to crack
  • AES-256: ~10^51 years (longer than universe age)

Status: INFEASIBLE
Mitigation: Use AES-128 minimum (AES-256 for high security)


Demonstration with 16-bit key (weak):
  Key space: 65,536 keys
  Time at 1M keys/sec: 0.07 seconds

✗ 16-bit key is INSECURE
✓ AES-256: 2^256 keys = SECURE


### Attack Method 2: Known-Plaintext Attack

In [62]:
print("ATTACK 2: KNOWN-PLAINTEXT ATTACK")
print("="*70)
print("""
Concept: Attacker has plaintext-ciphertext pairs, tries to deduce key

Example Scenario:
  • Attacker knows file format headers (e.g., PDF starts with %PDF)
  • Has encrypted version of same file
  • Attempts to derive key from known pairs

Against AES:
  • Theoretical attacks exist but require:
    - Billions of plaintext-ciphertext pairs
    - Enormous computational resources
  • No practical known-plaintext attack on full AES

Status: INFEASIBLE for proper AES implementation
Mitigation: Use proper modes (CBC, GCM) with random IV
""")

# Demonstrate why ECB mode is vulnerable
print("\nECB MODE VULNERABILITY (Pattern Leakage):")
print("-"*70)

repeated_data = b"AAAA" * 16  # Repeating pattern
key_ecb = get_random_bytes(32)

# ECB mode (insecure)
cipher_ecb = AES.new(key_ecb, AES.MODE_ECB)
encrypted_ecb = cipher_ecb.encrypt(repeated_data)

print(f"Plaintext (repeated): {repeated_data.hex()[:80]}...")
print(f"ECB Encrypted: {encrypted_ecb.hex()[:80]}...")
print("\n✗ Notice: Identical plaintext blocks → identical ciphertext blocks")
print("✗ Pattern leakage reveals information")

# CBC mode (secure)
cipher_cbc = AES.new(key_ecb, AES.MODE_CBC)
encrypted_cbc = cipher_cbc.encrypt(repeated_data)

print(f"\nCBC Encrypted: {encrypted_cbc.hex()[:80]}...")
print("CBC with random IV: No pattern leakage")

ATTACK 2: KNOWN-PLAINTEXT ATTACK

Concept: Attacker has plaintext-ciphertext pairs, tries to deduce key

Example Scenario:
  • Attacker knows file format headers (e.g., PDF starts with %PDF)
  • Has encrypted version of same file
  • Attempts to derive key from known pairs

Against AES:
  • Theoretical attacks exist but require:
    - Billions of plaintext-ciphertext pairs
    - Enormous computational resources
  • No practical known-plaintext attack on full AES

Status: INFEASIBLE for proper AES implementation
Mitigation: Use proper modes (CBC, GCM) with random IV


ECB MODE VULNERABILITY (Pattern Leakage):
----------------------------------------------------------------------
Plaintext (repeated): 41414141414141414141414141414141414141414141414141414141414141414141414141414141...
ECB Encrypted: 61994da49ca256a4640b0cd273bb622261994da49ca256a4640b0cd273bb622261994da49ca256a4...

✗ Notice: Identical plaintext blocks → identical ciphertext blocks
✗ Pattern leakage reveals information

C

### Attack Method 3: Side-Channel Attacks

In [63]:
print("ATTACK 3: SIDE-CHANNEL ATTACKS")
print("="*70)
print("""
Concept: Exploit physical implementation rather than algorithm itself

Types:
1. Timing Attacks
   • Measure encryption/decryption time
   • Different operations take different times
   • Can reveal key bits

2. Power Analysis
   • Measure power consumption during crypto operations
   • Different operations consume different power
   • Differential Power Analysis (DPA) can extract keys

3. Cache Timing Attacks
   • Exploit CPU cache behavior
   • S-box lookups may cause cache hits/misses
   • Can leak key information

4. Fault Injection
   • Induce errors (voltage glitches, radiation)
   • Analyze faulty outputs to deduce key

Mitigation:
  - Constant-time implementations
  - AES-NI hardware instructions
  - Secure hardware (HSM, TPM)
  - Randomized execution
  - Power supply filtering

Status: PRACTICAL in specific scenarios (physical access)
""")

# Demonstrate timing difference (simplified)
print("\nTiming Attack Demonstration (Simplified):")
print("-"*70)

key1 = get_random_bytes(32)
data = b"A" * 1000

times = []
for _ in range(5):
    start = time.perf_counter()
    cipher = AES.new(key1, AES.MODE_CBC)
    cipher.encrypt(pad(data, AES.block_size))
    times.append(time.perf_counter() - start)

print(f"Encryption times: {[f'{t*1000:.4f}ms' for t in times]}")
print("\n[Warning]: Timing variations could leak information")
print("Modern AES implementations use constant-time operations")

ATTACK 3: SIDE-CHANNEL ATTACKS

Concept: Exploit physical implementation rather than algorithm itself

Types:
1. Timing Attacks
   • Measure encryption/decryption time
   • Different operations take different times
   • Can reveal key bits

2. Power Analysis
   • Measure power consumption during crypto operations
   • Different operations consume different power
   • Differential Power Analysis (DPA) can extract keys

3. Cache Timing Attacks
   • Exploit CPU cache behavior
   • S-box lookups may cause cache hits/misses
   • Can leak key information

4. Fault Injection
   • Induce errors (voltage glitches, radiation)
   • Analyze faulty outputs to deduce key

Mitigation:
  - Constant-time implementations
  - AES-NI hardware instructions
  - Secure hardware (HSM, TPM)
  - Randomized execution
  - Power supply filtering

Status: PRACTICAL in specific scenarios (physical access)


Timing Attack Demonstration (Simplified):
--------------------------------------------------------------------

### Attack Method 4: Weak Key Derivation

In [64]:
print("ATTACK 4: WEAK KEY DERIVATION")
print("="*70)
print("""
Concept: Attack the key generation, not AES itself

Vulnerable Scenarios:
1. Weak passwords
   • User chooses "password123"
   • Easy to guess via dictionary attack

2. Poor random number generation
   • Using time.time() as seed
   • Predictable keys

3. Insufficient key derivation
   • Direct password → key without KDF
   • No salt, low iterations

Mitigation:
  - Strong passwords / passphrases
  - Proper KDF (PBKDF2, Argon2, scrypt)
  - Cryptographically secure RNG
  - Salt + high iteration count

Status: PRACTICAL if weak keys used
""")

# Bad: Direct password hash
weak_password = "password123"
weak_key = hashlib.sha256(weak_password.encode()).digest()
print("\n[BAD]: Direct hash as key")
print(f"  Password: {weak_password}")
print(f"  Key: {weak_key.hex()[:40]}...")
print("  Vulnerable to dictionary/rainbow table attacks")

salt = os.urandom(16)
kdf = PBKDF2HMAC(
    algorithm=hashes.SHA256(),
    length=32,
    salt=salt,
    iterations=100000,
    backend=default_backend()
)
strong_key = kdf.derive(weak_password.encode())

print("\n[GOOD]: PBKDF2 with salt")
print(f"  Salt: {salt.hex()}")
print("  Iterations: 100,000")
print(f"  Key: {strong_key.hex()[:40]}...")
print("  Much slower to brute force")

ATTACK 4: WEAK KEY DERIVATION

Concept: Attack the key generation, not AES itself

Vulnerable Scenarios:
1. Weak passwords
   • User chooses "password123"
   • Easy to guess via dictionary attack

2. Poor random number generation
   • Using time.time() as seed
   • Predictable keys

3. Insufficient key derivation
   • Direct password → key without KDF
   • No salt, low iterations

Mitigation:
  - Strong passwords / passphrases
  - Proper KDF (PBKDF2, Argon2, scrypt)
  - Cryptographically secure RNG
  - Salt + high iteration count

Status: PRACTICAL if weak keys used


[BAD]: Direct hash as key
  Password: password123
  Key: ef92b778bafe771e89245b89ecbc08a44a4e166c...
  Vulnerable to dictionary/rainbow table attacks

[GOOD]: PBKDF2 with salt
  Salt: b952d06148044d805ab96d254d2501b5
  Iterations: 100,000
  Key: f2d08f4947b9c00f41f2e4802b9d32b73a67018e...
  Much slower to brute force


## 5. Symmetric vs Asymmetric Encryption

### Comprehensive Comparison:

In [65]:
print("SYMMETRIC vs ASYMMETRIC ENCRYPTION")
print("="*70)

comparison = [
    ("Keys", "Single shared secret key", "Public key + Private key pair"),
    ("Key Distribution", "Difficult (secure channel needed)", "Easy (public key shared openly)"),
    ("Speed", "Very fast (1000x+ faster)", "Slow (computationally intensive)"),
    ("Key Size", "128-256 bits", "2048-4096 bits"),
    ("Algorithm", "AES, ChaCha20, 3DES", "RSA, ECC, ElGamal"),
    ("Use Case", "Bulk data encryption", "Key exchange, signatures"),
    ("Scalability", "O(n²) keys for n users", "O(n) keys for n users"),
    ("Data Size", "No practical limit", "Limited (~190 bytes for RSA-2048)"),
    ("Security Basis", "Key secrecy", "Mathematical hard problems"),
    ("Example", "File encryption, VPN", "HTTPS key exchange, email signing"),
]

print(f"\n{'Feature':<18} {'Symmetric (AES)':<35} {'Asymmetric (RSA)'}")
print("-"*100)
for feature, symmetric, asymmetric in comparison:
    print(f"{feature:<18} {symmetric:<35} {asymmetric}")

print("\n" + "="*70)

SYMMETRIC vs ASYMMETRIC ENCRYPTION

Feature            Symmetric (AES)                     Asymmetric (RSA)
----------------------------------------------------------------------------------------------------
Keys               Single shared secret key            Public key + Private key pair
Key Distribution   Difficult (secure channel needed)   Easy (public key shared openly)
Speed              Very fast (1000x+ faster)           Slow (computationally intensive)
Key Size           128-256 bits                        2048-4096 bits
Algorithm          AES, ChaCha20, 3DES                 RSA, ECC, ElGamal
Use Case           Bulk data encryption                Key exchange, signatures
Scalability        O(n²) keys for n users              O(n) keys for n users
Data Size          No practical limit                  Limited (~190 bytes for RSA-2048)
Security Basis     Key secrecy                         Mathematical hard problems
Example            File encryption, VPN                HTTPS

### Performance Comparison:

In [69]:
print("PERFORMANCE COMPARISON")
print("="*70)

# Test data
test_data = b"A" * 100

# Symmetric (AES)
aes_key = get_random_bytes(32)
start = time.perf_counter()
cipher_sym = AES.new(aes_key, AES.MODE_CBC)
enc_sym = cipher_sym.encrypt(pad(test_data, AES.block_size))
aes_time = time.perf_counter() - start

# Asymmetric (RSA)
rsa_private = rsa.generate_private_key(65537, 2048, default_backend())
rsa_public = rsa_private.public_key()
start = time.perf_counter()
enc_asym = rsa_public.encrypt(
    test_data,
    asym_padding.OAEP(
        mgf=asym_padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None
    )
)
rsa_time = time.perf_counter() - start

print(f"Data size: {len(test_data)} bytes\n")
print("AES-256 (Symmetric):")
print(f"  Time: {aes_time*1000:.4f}ms")
print(f"  Encrypted size: {len(enc_sym)} bytes\n")

print("RSA-2048 (Asymmetric):")
print(f"  Time: {rsa_time*1000:.4f}ms")
print(f"  Encrypted size: {len(enc_asym)} bytes\n")

print(f"Speed Ratio: RSA is ~{rsa_time/aes_time:.0f}x SLOWER than AES")
print("\nThis is why hybrid encryption (RSA + AES) is used in practice")

PERFORMANCE COMPARISON
Data size: 100 bytes

AES-256 (Symmetric):
  Time: 0.3834ms
  Encrypted size: 112 bytes

RSA-2048 (Asymmetric):
  Time: 0.2313ms
  Encrypted size: 256 bytes

Speed Ratio: RSA is ~1x SLOWER than AES

This is why hybrid encryption (RSA + AES) is used in practice


### Key Distribution Problem:

In [67]:
print("KEY DISTRIBUTION SCALABILITY")
print("="*70)

def keys_needed(n_users):
    symmetric = n_users * (n_users - 1) // 2
    asymmetric = n_users * 2
    return symmetric, asymmetric

print(f"\n{'Users':<10} {'Symmetric Keys':<20} {'Asymmetric Keys':<20} {'Ratio'}")
print("-"*70)

for n in [5, 10, 50, 100, 1000]:
    sym, asym = keys_needed(n)
    ratio = sym / asym
    print(f"{n:<10} {sym:<20,} {asym:<20} {ratio:.0f}x more")

print("\n✓ Asymmetric: Each user needs ONE key pair")
print("✗ Symmetric: Every user pair needs UNIQUE shared key")
print("\n→ This is why HTTPS uses asymmetric for key exchange,")
print("  then symmetric for actual data encryption")

KEY DISTRIBUTION SCALABILITY

Users      Symmetric Keys       Asymmetric Keys      Ratio
----------------------------------------------------------------------
5          10                   10                   1x more
10         45                   20                   2x more
50         1,225                100                  12x more
100        4,950                200                  25x more
1000       499,500              2000                 250x more

✓ Asymmetric: Each user needs ONE key pair
✗ Symmetric: Every user pair needs UNIQUE shared key

→ This is why HTTPS uses asymmetric for key exchange,
  then symmetric for actual data encryption


### When to Use Each:

In [70]:
print("DECISION GUIDE: WHICH ENCRYPTION TO USE?")
print("="*70)

scenarios = [
    ("Encrypting large files", "✓ Symmetric (AES)", "Fast, no size limit"),
    ("Database encryption", "✓ Symmetric (AES)", "Performance critical"),
    ("Disk/volume encryption", "✓ Symmetric (AES)", "Speed required"),
    ("VPN tunnel", "✓ Symmetric (AES)", "After key exchange"),
    ("Secure key exchange", "✓ Asymmetric (RSA/ECC)", "No pre-shared key needed"),
    ("Digital signatures", "✓ Asymmetric (RSA/ECC)", "Proves authenticity"),
    ("Email encryption (PGP)", "✓ Hybrid (RSA+AES)", "Best of both"),
    ("HTTPS/TLS", "✓ Hybrid (RSA+AES)", "Handshake + data"),
    ("Password storage", "✗ Neither", "Use hashing (Argon2)"),
]

print(f"\n{'Scenario':<25} {'Choice':<25} {'Reason'}")
print("-"*70)
for scenario, choice, reason in scenarios:
    print(f"{scenario:<25} {choice:<25} {reason}")

print("\n" + "="*70)
print("HYBRID APPROACH (Best Practice):")
print("  1. Use asymmetric (RSA) to exchange symmetric key")
print("  2. Use symmetric (AES) to encrypt actual data")
print("  3. Get security of RSA + speed of AES")
print("\nThis is what TLS/SSL, PGP, and S/MIME do!")

DECISION GUIDE: WHICH ENCRYPTION TO USE?

Scenario                  Choice                    Reason
----------------------------------------------------------------------
Encrypting large files    ✓ Symmetric (AES)         Fast, no size limit
Database encryption       ✓ Symmetric (AES)         Performance critical
Disk/volume encryption    ✓ Symmetric (AES)         Speed required
VPN tunnel                ✓ Symmetric (AES)         After key exchange
Secure key exchange       ✓ Asymmetric (RSA/ECC)    No pre-shared key needed
Digital signatures        ✓ Asymmetric (RSA/ECC)    Proves authenticity
Email encryption (PGP)    ✓ Hybrid (RSA+AES)        Best of both
HTTPS/TLS                 ✓ Hybrid (RSA+AES)        Handshake + data
Password storage          ✗ Neither                 Use hashing (Argon2)

HYBRID APPROACH (Best Practice):
  1. Use asymmetric (RSA) to exchange symmetric key
  2. Use symmetric (AES) to encrypt actual data
  3. Get security of RSA + speed of AES

This is what T

## Summary

### AES Overview:
- **Algorithm:** Block cipher with 128-bit blocks
- **Key sizes:** 128, 192, or 256 bits
- **Rounds:** 10-14 rounds of SubBytes, ShiftRows, MixColumns, AddRoundKey
- **Modes:** CBC (common), GCM (recommended), CTR, ECB (avoid)

### How AES Works:
1. Key expansion creates round keys
2. Initial AddRoundKey
3. Multiple rounds of 4 transformations
4. Final round (no MixColumns)
5. Output ciphertext

### Attack Scenarios:
1. **Brute Force:** Infeasible (2^256 keys for AES-256)
2. **Known-Plaintext:** No practical attack on full AES
3. **Side-Channel:** Practical with physical access
4. **Weak Keys:** Practical if poor key derivation

### Symmetric vs Asymmetric:

| Aspect | Symmetric (AES) | Asymmetric (RSA) |
|--------|-----------------|------------------|
| **Speed** | Very fast | Slow |
| **Key Mgmt** | Difficult | Easy |
| **Use Case** | Bulk data | Key exchange |
| **Best For** | Encryption | Authentication |

### Best Practices:
- Use AES-256 for sensitive data
- Always use CBC or GCM mode (never ECB)
- Use random IV for each encryption
- Proper key derivation (PBKDF2/Argon2)
- Consider authenticated encryption (GCM)
- Use hybrid approach for network communication